## Environments

In [1]:
!python -V

Python 3.7.7


In [2]:
import torch, torchvision
torch.__version__, torchvision.__version__

('1.6.0', '0.7.0')

In [3]:
%cd /workspace/src/mmrotate

/workspace/src/mmrotate


In [4]:
!rm -r /workspace/src/mmrotate/mmrotate.egg-info
!pip install -r requirements/build.txt
!pip install -v -e .

Using pip 23.1.2 from /opt/conda/lib/python3.7/site-packages/pip (python 3.7)
Obtaining file:///workspace/src/mmrotate
  Preparing metadata (setup.py) ...   Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-lxorgbzz/mmrotate.egg-info
  writing /tmp/pip-pip-egg-info-lxorgbzz/mmrotate.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-lxorgbzz/mmrotate.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-lxorgbzz/mmrotate.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-lxorgbzz/mmrotate.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-lxorgbzz/mmrotate.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-pip-egg-info-lxorgbzz/mmrotate.egg-info/SOURCES.txt'
  reading manifest template 'MANIFEST.in'
  adding license file 'LICENSE'
  writing manifest file '/tmp/pip-pip-egg-info-lxorgbzz/mmrotate.egg-info/SOURCES.txt'
done
  Attempting uninstall: mmrotate
  

In [5]:
# notebook directory
%cd /workspace/src

/workspace/src


In [6]:
import mmcv
import mmrotate ## mmcv>=1.3.17, <=1.8.0.
mmcv.__version__, mmrotate.__version__

/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


('1.7.1', '0.3.4')

## GPU Check

In [8]:
import torch 
# gpu check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

torch.cuda.is_available()

cuda


True

In [9]:
!nvidia-smi

Sat Jul  1 10:18:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:08:00.0  On |                  Off |
| 30%   61C    P0    97W / 300W |    551MiB / 49140MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Pretrained model

In [11]:
!apt install wget

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  wget
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 316 kB of archives.
After this operation, 954 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 wget amd64 1.19.4-1ubuntu2.2 [316 kB]
Fetched 316 kB in 1s (219 kB/s)3m
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package wget.
(Reading database ... 18025 files and directories currently installed.)
Preparing to unpack .../wget_1.19.4-1ubuntu2.2_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 17%] [#########.................................................] 8Unpacking wget (1.19.4-1ubuntu2.2) ...
7Progress: [ 33%] [###################.......................................] 87Progre

In [12]:
!wget https://download.openmmlab.com/mmrotate/v0.1.0/orientedreppoints/oriented_reppoints_r50_fpn_1x_dota_le135/oriented_reppoints_r50_fpn_1x_dota_le135-ef072de9.pth  -P ./weights/

--2023-06-30 14:12:26--  https://download.openmmlab.com/mmrotate/v0.1.0/orientedreppoints/oriented_reppoints_r50_fpn_1x_dota_le135/oriented_reppoints_r50_fpn_1x_dota_le135-ef072de9.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 128.1.157.210, 128.1.157.212, 128.1.157.213, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|128.1.157.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 147620842 (141M) [application/octet-stream]
Saving to: ‘./weights/oriented_reppoints_r50_fpn_1x_dota_le135-ef072de9.pth’

oriented_reppoints_ 100%[===================>] 140.78M  18.7MB/s    in 9.3s    

2023-06-30 14:12:39 (15.1 MB/s) - ‘./weights/oriented_reppoints_r50_fpn_1x_dota_le135-ef072de9.pth’ saved [147620842/147620842]



## DOTA v1

In [19]:
from pycocotools.coco import COCO
from tqdm import tqdm
import os
import cv2

COCO_ROOT = '/workspace/sample/train_scale1_h640_w640_oh0.0_ow0.0_min1'

In [47]:
for phase in ['train', 'valid']:
    
    COUNT_NO_LABEL = 0
    
    os.makedirs(COCO_ROOT + '/' + phase, exist_ok=True)
    coco = COCO(f'{COCO_ROOT}/{phase}.json')

    catIds = coco.getCatIds(catNms=['car']);
    imgIds = coco.getImgIds(catIds=catIds);
    
    print('\n' , f'{len(imgIds)} images for {phase}')
    
    for idx_img in tqdm(imgIds, total=len(imgIds)):
        try:
            img_info = coco.loadImgs(imgIds[idx_img])[0]
            annIds = coco.getAnnIds(imgIds=img_info['id'], catIds=catIds, iscrowd=None)
            anns = coco.loadAnns(annIds)
            
            DOTA_FMTS = []
            
            for ann in anns:
                bbox = ann['bbox']
                bbox = [
                    bbox[0], bbox[1], 
                    bbox[0] + bbox[2], bbox[1],
                    bbox[0] + bbox[2], bbox[1] + bbox[3],
                    bbox[0], bbox[1] + bbox[3],
                    ]
                bbox = list(map(str,map(round, bbox)))
                DOTA_FMTS.append(' '.join(bbox) + f' car 0')
                
            img = cv2.imread(f'{COCO_ROOT}/{img_info["file_name"]}')
            cv2.imwrite(f'{COCO_ROOT}/{phase}/{img_info["file_name"]}', img)
                
            # write to txet file
            with open(f'{COCO_ROOT}/{phase}/{img_info["file_name"]}.txt', 'w') as f:
                f.write('\n'.join(DOTA_FMTS))
        except Exception as e:
            COUNT_NO_LABEL += 1
            
    print(f'{COUNT_NO_LABEL} images have no label phase {phase}')
            

  8%|▊         | 9/110 [00:00<00:01, 87.36it/s]

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

 110 images for train


100%|██████████| 63/63 [00:00<00:00, 556.10it/s]

5 images have no label phase train
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

 63 images for valid
52 images have no label phase valid


In [50]:
!cp /workspace/src/dota.py /workspace/src/mmrotate/mmrotate/datasets/dota.py

## Train

In [51]:
!python mmrotate/tools/train.py /workspace/src/cfg/oriented_reppoints_basic.py

/opt/conda/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '
/workspace/src/mmrotate/mmrotate/utils/setup_env.py:39: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/workspace/src/mmrotate/mmrotate/utils/setup_env.py:49: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system be